In [ ]:
import fitz  # PyMuPDF
import re

# Define the key terms to look for based on the blood test markers observed in the PDF
test_terms = [
    "CPK", "AST", "ALT", "ALP", "GGT", "Bilirubina Tot", "Proteine Tot", "Albumine", 
    "Colesterolo", "Trigliceridi", "Globuline", "Rapp. Alb/Glob", "Amilasi", "Lipasi", 
    "UREA", "CREA", "Glucosio", "Calcio", "Fosforo", "Sodio", "Potassio", 
    "Rapp. Na/K", "Cloro", "Cloro corr.", "Lattato", "HCO3", "Div. anionico", 
    "Osmolarità cal.", "Ferro", "UIBC", "TIBC", "Saturazione", "Ferritina", 
    "Ac. Biliari post", "Alfa 2 Macroglobulina", "Aptoglobina HPT", "Ac. Biliari pre", 
    "Cistatina C", "Colinesterasi", "LDH", "Fruttosamina", "Magnesio", 
    "Calcio Ionico", "Ceruloplasmina", "Proteina C Reatt.", "AGP", "Acido Urico"
]

# Path to the PDF
pdf_path = '/mnt/data/_referti_10522430585_20241022_Assin_dicio_hope_Bio_24L4609.pdf'

def extract_blood_test_data(pdf_path):
    blood_test_data = {}

    # Open the PDF
    with fitz.open(pdf_path) as pdf:
        # Iterate over each page
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            text = page.get_text()

            # Look for each term in test_terms and extract nearby values
            for term in test_terms:
                # Create a regex pattern to find term followed by numbers
                pattern = rf"({term})\s*[:\-]?\s*([\d\.,]+)"
                matches = re.findall(pattern, text)

                # Store matches in the dictionary
                for match in matches:
                    param, value = match
                    blood_test_data[param] = value

    return blood_test_data

# Run the extraction function
blood_test_data = extract_blood_test_data(pdf_path)
print(blood_test_data)
